4. Tenemos dos grafos no dirigidos $G_1$ y $G_2$ , con la misma cantidad de vértices y aristas. $G_1$ es un grafo aleatorio de Erdös-Rényi, mientras $G_2$ es un grafo que cumple la ley de potencias en la distribución de los grados. Consideremos un virus que comienza en un único vértice aleatorio y se expande según el modelo **SIR**.

    a. ¿En cuál grafo es más probable que ocurra una epidemia (i.e. se infecte al menos un 30% de la red)? Justificar brevemente la respuesta. 

    b. Supongamos que en vez de comenzar en un vértice aleatorio, la epidemia comenzara en el vértice de mayor grado de $G_1$ y $G_2$ , respectivamente. ¿En cuál de los grafos es más probable que ocurra una epidemia? Justificar brevemente la respuesta.

    c. ¿Cómo afecta la existencia (o no existencia) de comunidades en la expansión de la epidemia?

    Para responder estas preguntas, se les recomienda realizar simulaciones. Pueden agregar todo tipo de resultados obtenidos para justificar sus respuestas.

**[3 Puntos]**

Funciones auxiliares

---

In [91]:
def infeccion_alcanzada(iteracion, porcentaje):
    '''
    Condicion de corte para la iteracion de la epidemia.
    En el caso que la iteración halla alcanzado el porcentaje de infeccion
    indicado, se retorna True.
    '''
    total_nodes = iteracion["node_count"][0]+iteracion["node_count"][1]+iteracion["node_count"][2]
    return (iteracion["node_count"][1]+iteracion["node_count"][2])/total_nodes >= (porcentaje/100)

In [92]:
import networkx as nx
import sys
sys.path.append('../')
from social_networks_utils.metricas import grado_promedio, clustering
from social_networks_utils.homofilia import contar_aristas

In [93]:
def graph_info_to_dict(G):
    '''
    Retorna un diccionario con la informacion 
    de las características principales del grafo G.
    '''
    d = {}
    d['Diametro'] = nx.diameter(G)
    d['Grado Promedio'] = grado_promedio(G)
    d['Coeficiente de clustering'] = clustering(G)[1]
    d['Cantidad de aristas'] = contar_aristas(G)
    return d

In [94]:
import numpy as np
import matplotlib.pyplot as plt

In [96]:
def graficar_histogramas_de_grados(g1,g2):
    '''
    Grafica los histogramas de los grados de los grafos g1 y g2.
    '''
    G = g1

    degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
    dmax = max(degree_sequence)

    fig = plt.figure("Degree of a random graph", figsize=(8, 8))
    # Create a gridspec for adding subplots of different sizes
    axgrid = fig.add_gridspec(6, 6)

    ax1 = fig.add_subplot(axgrid[:3, :])
    ax1.bar(*np.unique(degree_sequence, return_counts=True))
    ax1.set_title("Histograma de grados Preferential Attachment")
    ax1.set_xlabel("Degree")
    ax1.set_ylabel("# of Nodes")

    G = g2

    degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
    dmax = max(degree_sequence)

    ax2 = fig.add_subplot(axgrid[3:, :])
    ax2.bar(*np.unique(degree_sequence, return_counts=True))
    ax2.set_title("Histograma de grados Erdös-Rényi")
    ax2.set_xlabel("Degree")
    ax2.set_ylabel("# of Nodes")

    fig.tight_layout()
    plt.show()

---

In [2]:
from social_networks_utils.modelos import preferential_attachment, erdos_renyi

Primero creamos ambos grafos con cantidad similar de vertices y aristas

In [14]:
g1 = preferential_attachment(False, 2.5, 2000, 10)

In [15]:
len(g1.edges()), len(g1.nodes())

(26402, 2000)

In [26]:
g2 = erdos_renyi(2000, grado_promedio(g1))

In [27]:
len(g2.edges()), len(g2.nodes())

(26337, 2000)

In [ ]:
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc

In [40]:
model1 = ep.SIRModel(g1)
model2 = ep.SIRModel(g2)
# Model Configuration
# Datos obtenidos a parir del ejemplo de la documentación
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01)
cfg.add_model_parameter('gamma', 0.005)
cfg.add_model_parameter("fraction_infected", 0.05)

# Inicialiazamos los modelos con la configuracion que queremos
model1.set_initial_status(cfg)
model2.set_initial_status(cfg)

In [44]:
veces_infeccion_alcanzada = {}

In [45]:

model1.set_initial_status(cfg)
model2.set_initial_status(cfg)

for _ in range(4000):
    model1.reset()
    model2.reset()
    
    iteracion1 = model1.iteration()
    iteracion2 = model2.iteration()
    
    while((not infeccion_alcanzada(iteracion1, 30) and not infeccion_alcanzada(iteracion2, 30))):
        iteracion1 = model1.iteration()
        iteracion2 = model2.iteration()
    
    if(infeccion_alcanzada(iteracion1, 30)):
        veces_infeccion_alcanzada["model1"] = veces_infeccion_alcanzada.get("model1", 0) + 1
    if(infeccion_alcanzada(iteracion2, 30)):
        veces_infeccion_alcanzada["model2"] = veces_infeccion_alcanzada.get("model2", 0) + 1


In [46]:
veces_infeccion_alcanzada

{'model2': 2914, 'model1': 3042}

In [ ]:
import pandas as pd

In [90]:
df = pd.DataFrame([graph_info_to_dict(g1), graph_info_to_dict(g2)]).T
columns = ['Preferential Attachment','Erdös-Rényi']
df.columns = columns
df

,Preferential Attachment,Erdös-Rényi
Diametro,4.000000,4.000000
Grado Promedio,26.402000,26.337000
Coeficiente de clustering,0.012666,0.013623
Cantidad de aristas,26402.000000,26337.000000


a. Luego de hacer 4000 simulaciones en las cuales las iteraciones cortaban en el caso que al menos en un grafo haya ocurrido una epidemia ()
podemos ver que se logra que ocurra una epidemia de forma muy similar en ambos grafos. En el modelo que cumple con la ley de potencias ocurrio 3042 veces y en el de Erdös-Rényi ocurrio 2914 veces.

Además comparando las características principales (diametro, grado promedio, coeficiente de clustering) de los grafos podemos ver que son semejantes. Las características que considero mas importantes para que la epidemia ocurra son los bajos coeficientes de clustering (en una red real tienden a ser más altos) y el bajo diametro (caracteristica similar a las redes reales).

En conclusión la probabilidad que se genere una epidemia es similar y muy alta para estos tipos de redes.

b. En el caso que se comience en el vértice de mayor grado, el grafo que cumple con la ley de potencias va a ser el que más probabilidades tenga de ocurrir una epidemia. Esto se debe a que las leyes de potencia aparecen de la ventaja acumulativa, esto quiere decir que en el grafo se va a generar nodos con grados muy altos y nodos con muy grados bajos. Tiene un balance desproporcionado.

Entonces es más probable el nodo de mayor grado se genere en el grafo que cumple con la ley de potencias y no en el Erdös-Rényi. Mientras mayor sea el grado del nodo, mayor alcance a otros nodos, generando una probabilidad que ocurra una epidemia más alta.

c. La existencia de comunidades no ayuda a la expansion de una epidemia, dado que se generan lazos débiles entre ellas. Si dos nodos conectan dos comunidades a través de un lazo debil y el infectado no logra contagiar al nodo de la otra comunidad, ninguno dentro de la comunidad 'sana' va a poder contagiarse.